<a href="https://colab.research.google.com/github/vrra/FGAN-Build-a-thon/blob/main/PythonScripts2023/Spacy-example-01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install -U spacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 929.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 254.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 297.1 kB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.6.1
    Uninstalling spacy-3.6.1:
      Successfully uninstalled spacy-3.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.6.0 requires spacy<3.7.0,>=3.6.0, but you have spacy 3.7.2 which is incompatible.


In [2]:
pip install textacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.7/210.7 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.4/320.4 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 1.5 MB/s eta 0:00:00


In [3]:
import spacy


In [4]:
import textacy


In [5]:
nlp = spacy.load("en_core_web_sm")

/usr/local/lib/python3.10/dist-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_sm' (3.6.0) was trained with spaCy v3.6.0 and may not be 100% compatible with the current version (3.7.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [62]:
sentences = ["All living things are made of cells.",
             "Aaron is a good boy.",
             "cells  have organelles",
             "Kennedy lives in UK.",
             "Prof James is in town",
             "Othniel is having exams today"]

In [68]:
verb_patterns = [[{"POS":"AUX"}, {"POS":"VERB"}, {"POS":"ADP"}],
                 [{"POS":"AUX"}, {"POS":"ADJ"}],
                 [{"POS":"VERB"}],
                 [{"POS":"AUX"}]
                ]

In [20]:

def contains_root(verb_phrase, root):
    vp_start = verb_phrase.start
    vp_end = verb_phrase.end
    if (root.i >= vp_start and root.i <= vp_end):
        return True
    else:
        return False

In [21]:
def find_root_of_sentence(doc):
    root_token = None
    for token in doc:
        if (token.dep_ == "ROOT"):
            root_token = token
    return root_token

In [22]:
def get_verb_phrases(doc):
    root = find_root_of_sentence(doc)
    print(root)
    verb_phrases= textacy.extract.matches.token_matches(doc,verb_patterns)

    new_vps = []
    for verb_phrase in verb_phrases:
        if (contains_root(verb_phrase, root)):
            new_vps.append(verb_phrase)
    return new_vps

In [23]:
def longer_verb_phrase(verb_phrases):
    longest_length = 0
    longest_verb_phrase = None
    for verb_phrase in verb_phrases:
        if len(verb_phrase) > longest_length:
            longest_verb_phrase = verb_phrase
    return longest_verb_phrase

In [24]:
def find_noun_phrase(verb_phrase, noun_phrases, side):
    for noun_phrase in noun_phrases:
        if (side == "left" and \
            noun_phrase.start < verb_phrase.start):
            return noun_phrase
        elif (side == "right" and \
              noun_phrase.start > verb_phrase.start):
            return noun_phrase

In [25]:
def find_triplet(sentence):
    doc = nlp(sentence)
    verb_phrases = get_verb_phrases(doc)
    noun_phrases = doc.noun_chunks
    verb_phrase = None

    if (len(verb_phrases) > 1):
        verb_phrase = \
        longer_verb_phrase(list(verb_phrases))
    elif (len(verb_phrases)==1):
        verb_phrase = verb_phrases[0]
    elif (len(verb_phrases)==0):
        print("verb_phrases is NULL")
        return("","","")

    left_noun_phrase = find_noun_phrase(verb_phrase,
                                        noun_phrases,
                                        "left")
    right_noun_phrase = find_noun_phrase(verb_phrase,
                                         noun_phrases,
                                         "right")
    return (left_noun_phrase, verb_phrase,
            right_noun_phrase)

In [69]:
for sentence in sentences:
    (left_np, vp, right_np) = find_triplet(sentence)
    print("Triplets = ",left_np, "\t", vp, "\t", right_np)

made
Triplets =  All living things 	 are made of 	 cells
is
Triplets =  Aaron 	 is 	 a good boy
have
Triplets =  cells 	 have 	 organelles
lives
Triplets =  Kennedy 	 lives 	 UK
is
Triplets =  Prof James 	 is 	 town
having
Triplets =  Othniel 	 having 	 exams
